# Testing sklearn and statsmodel using artificial data

1. [Testing how `sklearn` performs with high collinearity](#testing-how-sklearn-performs-with-high-collinearity)
1. [Testing `StandardScaler`](#testing-standardscaler)
1. [Testing ridge regression](#testing-ridge-regression)
1. [Testing LASSO](#testing-lasso)
1. [Testing `statsmodel`](#testing-statsmodel)

In [81]:
# import modules

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import sklearn.linear_model as skl_lm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# ridge and lasso
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV

# statsmodel
import statsmodels.api as sm
from statsmodels.stats.outliers_influence \
    import variance_inflation_factor as VIF
from statsmodels.stats.anova import anova_lm
# from ISLP import load_data
from ISLP.models import (ModelSpec as MS, summarize, poly)
from scipy import stats

---

## Testing how `sklearn` performs with high collinearity

We generate a dataframe with 3 predictors:
$$Y=X_1 +2X_2 + 3X_3,$$

where $X_1$ and $X_2$ are strongly correlated with the relationship $X_2=2X_1$.

Results: 
- uses pseudoinverse
- doesn't give warning
- good fit when predictors are not scaled, expected confounding effect when scaled
- "equal weightage distribution" are due to scaler, not OLS function

In [116]:
# set parameters as you like to play around 
beta1=1
beta2=2
beta3=3
correlation=3 # X_2 = 2*X_1

# Define the number of rows and columns
num_rows = 100
num_cols = 4

# Define column names (optional, but good practice)
column_names = [f'col_{i+1}' for i in range(num_cols)]

# Generate random data using NumPy
# For random integers: np.random.randint(low, high, size=(rows, cols))
# For random floats: np.random.rand(rows, cols) or np.random.uniform(low, high, size=(rows, cols))
random_data = np.random.randint(0, 100, size=(num_rows, num_cols)) # Example: random integers between 0 and 99

# Create the DataFrame
df = pd.DataFrame(random_data, columns=column_names)

for i in range(num_rows):
    df.iloc[i,1]=correlation*df.iloc[i,0]#+ np.random.randint(-50,50)
    df.iloc[i,num_cols-1] = beta1*df.iloc[i,0] + beta2*df.iloc[i,1] + beta3*df.iloc[i,2] + np.random.randint(-10,10)

# Print the generated DataFrame
print(df)

    col_1  col_2  col_3  col_4
0      59    177     39    526
1      13     39     16    131
2      51    153     83    610
3      88    264     21    688
4      52    156     14    400
..    ...    ...    ...    ...
95     59    177     12    448
96     85    255     49    750
97      8     24     55    229
98     88    264     63    797
99     21     63     88    408

[100 rows x 4 columns]


In [125]:
X=df.drop(columns=['col_4'])
y=df['col_4']

X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.8,random_state=42)

# standardise
scaler=StandardScaler()
X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.transform(X_test)

# build model
model=LinearRegression()
model_scaled=model.fit(X_train,y_train)

# make predictions
y_pred1=model.predict(X_test_scaled)

print(model_scaled.coef_, model_scaled.intercept_)


[0.70082786 2.10248359 3.02729549] -2.055773247245895


/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


Real relationship:

$$Y=X_1+2X_2+3X_3$$

In [84]:
# equally distributed weightage?

b1=(model_scaled.coef_[1]+model_scaled.coef_[0])

print(b1)

216.96334945181675


In [124]:
# pseudoinverse for scaled data
pinv_arr=np.ones((int(0.8*num_rows),num_cols))
for i in range(X_train_scaled.shape[0]):
    for j in range(4):
        if j<3:
            pinv_arr[i,j]=X_train_scaled[i,j]

pinv=np.linalg.pinv(pinv_arr)
print(np.matmul(pinv,y_train))

[100.11755932 100.11755932  87.15896507 464.8375    ]


### What if we remove the second (collinear) column?

Now, there is no collinearity in our model, and we see that the coefficient for the first predictor is the sum of the coefficients of the two correlated variables in the previous model.

In [86]:
X_new=X.drop(columns=['col_2'])

X_new_train, X_new_test, y_train, y_test = train_test_split(X_new,y,train_size=0.8,random_state=42)

# standardise
X_new_train_scaled=scaler.fit_transform(X_new_train)
X_new_test_scaled=scaler.transform(X_new_test)

# build model
model_new=LinearRegression().fit(X_new_train_scaled,y_train)

# test
y_pred2=model_new.predict(X_new_test_scaled)

print(model_new.coef_, model_new.intercept_)

[216.96334945  78.58886962] 481.075


In [87]:
# comparing R2 values
print("with second column: ", r2_score(y_test,y_pred1))
print(" w/o second column: ", r2_score(y_test,y_pred2))

with second column:  0.930467431785162
 w/o second column:  0.9304674317851621


## Testing StandardScaler

### 1. What if we don't scale the regressors?

In [88]:
# model fit to unscaled data

model_unscaled=LinearRegression()
model_unscaled.fit(X_train,y_train)
print(model_unscaled.coef_, model_unscaled.intercept_)

# real relationship: Y=X_1+2X_2+3X_3

[0.6904844  2.07145319 2.58660988] 21.490577191873


In [89]:
# using pseudoinverse for unscaled data matrix

# add one column of ones for constant term
pinv_arr=np.ones((80,4))
for i in range(80):
    for j in range(4):
        if j<3:
            pinv_arr[i,j]=X_train.iloc[i,j]

pinv2=np.linalg.pinv(pinv_arr)
print(np.matmul(pinv2,y_train))

[ 0.6904844   2.07145319  2.58660988 21.49057719]


In [90]:
X_train.head

<bound method NDFrame.head of     col_1  col_2  col_3
55     18     54     96
88     48    144     65
26     53    159     81
42      4     12     27
69     74    222     41
..    ...    ...    ...
60     97    291     95
71     83    249     12
14     14     42     20
92     86    258     38
51     27     81     46

[80 rows x 3 columns]>

In [91]:
X_train_scaled

array([[-9.85379477e-01, -9.85379477e-01,  1.61356882e+00],
       [-3.06314977e-02, -3.06314977e-02,  5.93260132e-01],
       [ 1.28493166e-01,  1.28493166e-01,  1.11987107e+00],
       [-1.43092853e+00, -1.43092853e+00, -6.57440840e-01],
       [ 7.96816751e-01,  7.96816751e-01, -1.96656271e-01],
       [ 1.08324114e+00,  1.08324114e+00,  1.54774245e+00],
       [-9.85379477e-01, -9.85379477e-01,  5.60346949e-01],
       [-1.43092853e+00, -1.43092853e+00,  1.05404470e+00],
       [ 9.55941414e-01,  9.55941414e-01,  4.28694215e-01],
       [ 5.74042223e-01,  5.74042223e-01, -1.24987814e+00],
       [ 6.48433003e-02,  6.48433003e-02, -6.24527656e-01],
       [ 5.10392357e-01,  5.10392357e-01, -1.28279133e+00],
       [ 1.60318098e-01,  1.60318098e-01,  1.32475563e-01],
       [-9.21729612e-01, -9.21729612e-01, -1.18405178e+00],
       [ 1.46514034e+00,  1.46514034e+00,  7.90739233e-01],
       [ 1.19343497e-03,  1.19343497e-03, -1.30829904e-01],
       [-6.24564303e-02, -6.24564303e-02

Let's compare $\mathbf{X}$ before and after scaling 

In [92]:
# trying to standardise a column manually``

X_new=X_train
mean1=np.mean(X_train.iloc[:,0])
var1=np.var(X_train.iloc[:,0])
sd1=var1**0.5
X_new.iloc[:,0]=(X_train.iloc[:,0] - mean1 )/sd1
print(mean1,var1)
X_new.iloc[:,0]

48.9625 987.3360937500007


/var/folders/n1/xb4gyj5530g0114s2tcl_l_c0000gn/T/ipykernel_18612/654684410.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '55   -0.985379
88   -0.030631
26    0.128493
42   -1.430929
69    0.796817
        ...   
60    1.528790
71    1.083241
14   -1.112679
92    1.178716
51   -0.698955
Name: col_1, Length: 80, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X_new.iloc[:,0]=(X_train.iloc[:,0] - mean1 )/sd1


55   -0.985379
88   -0.030631
26    0.128493
42   -1.430929
69    0.796817
        ...   
60    1.528790
71    1.083241
14   -1.112679
92    1.178716
51   -0.698955
Name: col_1, Length: 80, dtype: float64

### 2. Testing the scaler with non-collinear data

Results: everything normal and consistent. The comparison between output of scaled and unscaled model is as expected.

In [93]:
# Define the number of rows and columns
num_rows = 100
num_cols = 4

# Define column names (optional, but good practice)
column_names = [f'col_{i+1}' for i in range(num_cols)]

# Generate random data using NumPy
# For random integers: np.random.randint(low, high, size=(rows, cols))
# For random floats: np.random.rand(rows, cols) or np.random.uniform(low, high, size=(rows, cols))

random_data = np.random.randint(0, 100, size=(num_rows, num_cols)) # Example: random integers between 0 and 99

# Create the DataFrame
df = pd.DataFrame(random_data, columns=column_names)

for i in range(num_rows):
    df.iloc[i,num_cols-1]=df.iloc[i,0]+2*df.iloc[i,1]+3*df.iloc[i,2]+np.random.randint(-10,10)

# Print the generated DataFrame
print(df)

    col_1  col_2  col_3  col_4
0      70     21     46    255
1      29     32     35    197
2      58     90     79    466
3      74      8     95    377
4      73     11     13    133
..    ...    ...    ...    ...
95      9     32     63    260
96      8     94     55    369
97     66     33      6    151
98     95     77     75    472
99     57     12     90    356

[100 rows x 4 columns]


In [94]:
X=df.drop(columns=['col_4'])
y=df['col_4']

X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.8,random_state=42)

# standardise
scaler=StandardScaler()
X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.transform(X_test)

# correct
model1=LinearRegression()
model2=LinearRegression()
model_scaled=model1.fit(X_train_scaled,y_train)
model_unscaled=model2.fit(X_train,y_train)

# wrong
# model=LinearRegression()
# model_scaled=model.fit(X_train_scaled,y_train)
# model_unscaled=model.fit(X_train,y_train)


In [95]:
df

,col_1,col_2,col_3,col_4
0,70,21,46,255
1,29,32,35,197
2,58,90,79,466
3,74,8,95,377
4,73,11,13,133
...,...,...,...,...
95,9,32,63,260
96,8,94,55,369
97,66,33,6,151
98,95,77,75,472


In [96]:
X_train_scaled

array([[ 1.3361366 ,  1.54383411, -0.0812089 ],
       [-1.50781411, -1.47954388,  1.07143349],
       [ 1.6135952 , -1.13202917,  0.02357678],
       [-1.50781411,  1.75234293,  0.79200503],
       [ 0.53844311, -0.88876887,  0.12836245],
       [ 1.40550125, -0.29799387, -1.61806541],
       [-0.88353225, -0.92352035,  1.59536185],
       [-1.54249644,  1.61333705,  0.05850533],
       [ 1.26677195,  0.77930174,  0.86186215],
       [-1.05694388, -0.40224828, -0.91949427],
       [-1.02226155, -0.99302329,  1.42071906],
       [ 0.95463102,  0.81405321,  0.33793379],
       [-1.47313179, -0.81926593,  0.23314812],
       [-0.32861504, -1.27103506, -0.46542302],
       [-1.75059039, -0.61075711, -0.18599457],
       [-0.84884992,  0.57079291,  0.86186215],
       [-0.05115643, -0.61075711,  0.61736225],
       [-1.36908481,  0.39703556, -0.81470859],
       [ 1.4748659 , -0.81926593,  1.42071906],
       [-0.46734434,  1.33532528, -0.3257088 ],
       [ 1.57891288,  0.39703556, -0.779

In [97]:
print("Scaled: ",model_scaled.coef_, model_scaled.intercept_)
print("Unscaled: ",model_unscaled.coef_, model_unscaled.intercept_)

Scaled:  [27.56749648 57.72438096 87.1780837 ] 307.0875
Unscaled:  [0.95610489 2.00600716 3.04500468] -0.8947691429685278


## Testing ridge regression

What I want to know: whether if it just gives a model that is useful for predictions OR it is actually able to capture the true underlying relationship. 

We reuse the same generated dataframe from last section, with relationship
$$Y=X_1+2X_2+3X_3$$
where $X_1$ and $X_2$ are correlated with $X_2=2X_1$.

In [98]:
# reuse the data
print(df)


    col_1  col_2  col_3  col_4
0      70     21     46    255
1      29     32     35    197
2      58     90     79    466
3      74      8     95    377
4      73     11     13    133
..    ...    ...    ...    ...
95      9     32     63    260
96      8     94     55    369
97     66     33      6    151
98     95     77     75    472
99     57     12     90    356

[100 rows x 4 columns]


In [99]:
X=df.iloc[:,0:3]
y=df.iloc[:,3]

X_train, X_test, y_train, y_test=train_test_split(X,y,train_size=0.8, random_state=42)

scaler=StandardScaler()
X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.transform(X_test)

ols=LinearRegression().fit(X_train_scaled,y_train)
ridge_cv=RidgeCV(alphas=[0.0001,0.001,0.01,0.1,1.0,5.0,10.0],cv=10) #alpha = regularisation parameter
ridge_cv.fit(X_train_scaled, y_train)

# make predictions
y_pred_ols=ols.predict(X_test_scaled)
y_pred_ridge=ridge_cv.predict(X_test_scaled)
print("Model score (R^2) for OLS: ", r2_score(y_test,y_pred_ols))
print("Model score (R^2) for ridge: ", r2_score(y_test,y_pred_ridge))



Model score (R^2) for OLS:  0.9968497537026608
Model score (R^2) for ridge:  0.996818497465305


In [100]:
from sklearn.model_selection import cross_val_score, KFold

# define cross-validation strategy
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# compute cross-validated R^2 scores
cv_scores = cross_val_score(ols, X_train_scaled, y_train, cv=cv, scoring='r2')

print("Cross-validated R^2 scores (OLS):", cv_scores)
print("Mean CV R^2 (OLS):", np.mean(cv_scores))

Cross-validated R^2 scores (OLS): [0.99737051 0.9985115  0.9950742  0.9979788  0.99781831]
Mean CV R^2 (OLS): 0.9973506658452187


In [101]:
ols.coef_

array([27.56749648, 57.72438096, 87.1780837 ])

In [102]:
ridge_cv.coef_

array([27.54383888, 57.65719607, 87.07138788])

In [103]:
ridge_cv.alpha_

0.1

## Testing LASSO 

(same as ridge)
What I want to know: whether if it just gives a model that is useful for predictions OR it is actually able to capture the true underlying relationship. 

We reuse the same generated dataframe from previous sections, with relationship
$$Y=X_1+2X_2+3X_3$$
where $X_1$ and $X_2$ are correlated with $X_2=2X_1$.

In [104]:
#Lasso Cross validation
lasso_cv = LassoCV(alphas = [0.0001, 0.001,0.01, 0.1, 1, 10], random_state=42).fit(X_train_scaled, y_train)

y_pred_lasso=lasso_cv.predict(X_test_scaled)
#score
# print(lasso_cv.score(X_train, y_train))
# print(lasso_cv.score(X_test, y_test))
r2_lasso=r2_score(y_test,y_pred_lasso)

# compare
print("Model score (R^2) for OLS:   ", r2_score(y_test,y_pred_ols))
print("Model score (R^2) for ridge: ", r2_score(y_test,y_pred_ridge))
print("Model score (R^2) for lasso: ", r2_lasso)
print(f'Alpha selected out of {lasso_cv.alphas}: {lasso_cv.alpha_}')

Model score (R^2) for OLS:    0.9968497537026608
Model score (R^2) for ridge:  0.996818497465305
Model score (R^2) for lasso:  0.996792805178038
Alpha selected out of [0.0001, 0.001, 0.01, 0.1, 1, 10]: 0.1


In [105]:
print("Test MSE for OLS:   ", mean_squared_error(y_test,y_pred_ols))
print("Test MSE for ridge: ", mean_squared_error(y_test,y_pred_ridge))
print("Test MSE for lasso: ", mean_squared_error(y_test,y_pred_lasso))

Test MSE for OLS:    30.742277039381314
Test MSE for ridge:  31.04729697030269
Test MSE for lasso:  31.29802003713197


## Testing statsmodel

In [126]:
X_train = sm.add_constant(X_train)
model_OLS=sm.OLS(y_train,X_train)
results_OLS=model_OLS.fit()
summarize(results_OLS)

,coef,std err,t,P>|t|
const,-2.0558,1.605,-1.281,0.204
col_1,0.7008,0.002,295.957,0.000
col_2,2.1025,0.007,295.957,0.000
col_3,3.0273,0.023,128.825,0.000


In [115]:
results_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  col_4   R-squared (uncentered):                   1.000
Model:                            OLS   Adj. R-squared (uncentered):              1.000
Method:                 Least Squares   F-statistic:                          3.687e+05
Date:                Tue, 26 Aug 2025   Prob (F-statistic):                   8.90e-156
Time:                        19:46:00   Log-Likelihood:                         -253.27
No. Observations:                  80   AIC:                                      510.5
Df Residuals:                      78   BIC:                                      515.3
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
col_1          0.7009      0.002    453.152      0.000       0.698       0.704
col_2          2.1028      0.005    453.152      0.000       2.094       2.112
col_3          2.9919      0.017    174.020      0.000       2.958       3.026
==============================================================================
Omnibus:                       23.316   Durbin-Watson:                   2.076
Prob(Omnibus):                  0.000   Jarque-Bera (JB):                4.980
Skew:                           0.139   Prob(JB):                       0.0829
Kurtosis:                       1.810   Cond. No.                     5.63e+15
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The smallest eigenvalue is 9.78e-26. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [131]:
results_OLS.condition_number

2.383061614115309e+16

In [128]:
model.coef_, model.intercept_

(array([0.70082786, 2.10248359, 3.02729549]), -2.055773247245895)

In [127]:
summarize(results_OLS)

,coef,std err,t,P>|t|
const,-2.0558,1.605,-1.281,0.204
col_1,0.7008,0.002,295.957,0.000
col_2,2.1025,0.007,295.957,0.000
col_3,3.0273,0.023,128.825,0.000


In [2]:
# ridge using statsmodel
ridge_model=model_OLS.fit_regularized(method='elastic_net',alpha=10, L1_wt=0.0)
print(ridge_model.params)


NameError: name 'model_OLS' is not defined

In [ ]:
# lasso using statsmodel